In [5]:

def is_valid_computer_move(grid, x, y):
    """Check if the computer move (horizontal placement) is valid."""
    return x < n - 1 and grid[y][x] == 0 and grid[y][x + 1] == 0


In [7]:
import pygame
import random
import math

# Constants
GRID_ROWS = 2

# Get user input
n = int(input("Enter the number of columns for the 2 × n Domineering game: "))

# Initialize Pygame
pygame.init()

# Screen setup
screen_info = pygame.display.Info()
window_width = screen_info.current_w
window_height = int(screen_info.current_h * 0.90)
screen = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption(f"2 × {n} Domineering Game")

# Grid calculations
CELL_SIZE = min(int(window_width * 0.8) // n, int(window_height * 0.6) // GRID_ROWS)
grid_width = n * CELL_SIZE
grid_height = GRID_ROWS * CELL_SIZE
offset_x = (window_width - grid_width) // 2
offset_y = (window_height - grid_height) // 2

# Game state
grid = [[0] * n for _ in range(GRID_ROWS)]
FONT_SIZE = int(window_height * 0.1)
font = pygame.font.Font(None, FONT_SIZE)
game_over = False
winner_message = ""
end_time = 0

# Strategy variables
strategy_phase = 0
computer_moves = []
initial_human_col = -1
initial_computer_moves = []
stack_index = 0

# AI parameters
SEARCH_DEPTH = 3 if n <= 8 else 2  # For general case

def simulate_move(old_grid, x, y, player):
    new_grid = [row.copy() for row in old_grid]
    if player == 1:  # Vertical
        new_grid[y][x] = 1
        new_grid[y+1][x] = 1
    else:  # Horizontal
        new_grid[y][x] = 2
        new_grid[y][x+1] = 2
    return new_grid

def evaluate_position(grid):
    computer_moves = sum(1 for y in range(GRID_ROWS) for x in range(n-1)
                        if is_valid_computer_move(grid, x, y))
    human_moves = sum(1 for x in range(n) for y in range(GRID_ROWS-1)
                     if is_valid_human_move(grid, x, y))
    center = n//2
    center_weight = sum(grid[y][x] == 2
                       for x in [center-1, center] if 0 <= x < n
                       for y in range(GRID_ROWS))
    return (computer_moves - 2*human_moves) + 0.5*center_weight

def minimax(grid, depth, alpha, beta, maximizing_player):
    if depth == 0 or check_terminal(grid, maximizing_player):
        return evaluate_position(grid)
    
    if maximizing_player:
        max_eval = -math.inf
        for y in range(GRID_ROWS):
            for x in range(n-1):
                if is_valid_computer_move(grid, x, y):
                    new_grid = simulate_move(grid, x, y, 2)
                    eval = minimax(new_grid, depth-1, alpha, beta, False)
                    max_eval = max(max_eval, eval)
                    alpha = max(alpha, eval)
                    if beta <= alpha:
                        break
        return max_eval
    else:
        min_eval = math.inf
        for x in range(n):
            for y in range(GRID_ROWS-1):
                if is_valid_human_move(grid, x, y):
                    new_grid = simulate_move(grid, x, y, 1)
                    eval = minimax(new_grid, depth-1, alpha, beta, True)
                    min_eval = min(min_eval, eval)
                    beta = min(beta, eval)
                    if beta <= alpha:
                        break
        return min_eval

def check_terminal(grid, computer_turn):
    if computer_turn:
        return not any(is_valid_computer_move(grid, x, y)
                      for y in range(GRID_ROWS) for x in range(n-1))
    else:
        return not any(is_valid_human_move(grid, x, y)
                      for x in range(n) for y in range(GRID_ROWS-1))

def handle_computer_move(grid):
    global strategy_phase, computer_moves, initial_human_col, initial_computer_moves, stack_index
    
    # Special strategies for n=12-14
    if n in [12, 13, 14]:
        # n=12 strategy
        if n == 12:
            if strategy_phase == 0:
                for x in range(n):
                    for y in range(GRID_ROWS-1):
                        if grid[y][x] == 1 and grid[y+1][x] == 1:
                            initial_human_col = x
                            strategy_phase = 1
                            break
                    if initial_human_col != -1:
                        break

            if strategy_phase == 1:
                target_col = 8 if initial_human_col <= 5 else 2
                if is_valid_computer_move(grid, target_col, 1):
                    make_computer_move(grid, target_col, 1)
                    strategy_phase = 2
                    return True

            elif strategy_phase == 2:
                first_move_col = computer_moves[0][0]
                left_side = first_move_col - 2
                right_side = first_move_col + 2
                if left_side >= 0 and is_valid_computer_move(grid, left_side, 1):
                    make_computer_move(grid, left_side, 1)
                    strategy_phase = 3
                    return True
                elif right_side+1 < n and is_valid_computer_move(grid, right_side, 1):
                    make_computer_move(grid, right_side, 1)
                    strategy_phase = 3
                    return True

            elif strategy_phase == 3:
                for x in range(n-1):
                    if is_valid_computer_move(grid, x, 1) and (x, 1) not in computer_moves:
                        make_computer_move(grid, x, 1)
                        strategy_phase = 4
                        return True

            elif strategy_phase == 4:
                last_move_col = computer_moves[-1][0]
                if is_valid_computer_move(grid, last_move_col, 0):
                    make_computer_move(grid, last_move_col, 0)
                    strategy_phase = 5
                    return True

            elif strategy_phase == 5:
                for move in computer_moves:
                    if move[1] == 1:
                        col = move[0]
                        if col+2 < n and is_valid_computer_move(grid, col+1, 0):
                            make_computer_move(grid, col+1, 0)
                            strategy_phase = 6
                            return True

        # n=13/14 strategy
        elif n in [13, 14]:
            if strategy_phase == 0:
                if len(initial_computer_moves) < 3:
                    for x in range(n-1):
                        if is_valid_computer_move(grid, x, 1) and (x, 1) not in initial_computer_moves:
                            make_computer_move(grid, x, 1)
                            initial_computer_moves.append((x, 1))
                            if len(initial_computer_moves) == 3:
                                strategy_phase = 1
                                stack_index = 0
                            return True
                    strategy_phase = 2

            elif strategy_phase == 1:
                if stack_index < len(initial_computer_moves):
                    x, y = initial_computer_moves[stack_index]
                    if is_valid_computer_move(grid, x, 0):
                        make_computer_move(grid, x, 0)
                        stack_index += 1
                        return True
                    else:
                        stack_index += 1
                else:
                    strategy_phase = 2

        # Common fallback for special n values
        if strategy_phase in [2, 6]:
            center = n // 2
            for offset in range(center + 1):
                for side in [center - offset, center + offset]:
                    if side >= 0 and side + 1 < n:
                        for y in range(GRID_ROWS):
                            if is_valid_computer_move(grid, side, y):
                                make_computer_move(grid, side, y)
                                return True
            
            for y in range(GRID_ROWS):
                for x in range(n - 1):
                    if is_valid_computer_move(grid, x, y):
                        make_computer_move(grid, x, y)
                        return True

    # General case minimax strategy
    else:
        best_score = -math.inf
        best_moves = []
        
        for y in range(GRID_ROWS):
            for x in range(n-1):
                if is_valid_computer_move(grid, x, y):
                    new_grid = simulate_move(grid, x, y, 2)
                    score = minimax(new_grid, SEARCH_DEPTH, -math.inf, math.inf, False)
                    
                    if score > best_score:
                        best_score = score
                        best_moves = [(x, y)]
                    elif score == best_score:
                        best_moves.append((x, y))
        
        if best_moves:
            x, y = random.choice(best_moves)
            make_computer_move(grid, x, y)
            return True
    
    return False

def make_computer_move(grid, x, y):
    grid[y][x] = 2
    grid[y][x+1] = 2
    computer_moves.append((x, y))

def is_valid_human_move(grid, x, y):
    return y < GRID_ROWS-1 and grid[y][x] == 0 and grid[y+1][x] == 0

def handle_human_move(grid, x, y):
    if is_valid_human_move(grid, x, y):
        grid[y][x] = 1
        grid[y+1][x] = 1
        return True
    else:
        msg = font.render("Invalid move!", True, (255, 0, 0))
        screen.blit(msg, msg.get_rect(center=(window_width//2, window_height//2)))
        pygame.display.flip()
        pygame.time.delay(500)
        return False

def draw_grid(screen):
    for x in range(n + 1):
        pygame.draw.line(screen, (0, 0, 0), 
                         (offset_x + x*CELL_SIZE, offset_y),
                         (offset_x + x*CELL_SIZE, offset_y + GRID_ROWS*CELL_SIZE), 1)
    
    for y in range(GRID_ROWS + 1):
        pygame.draw.line(screen, (0, 0, 0),
                         (offset_x, offset_y + y*CELL_SIZE),
                         (offset_x + n*CELL_SIZE, offset_y + y*CELL_SIZE), 1)
    
    for x in range(n):
        for y in range(GRID_ROWS):
            rect = pygame.Rect(offset_x + x*CELL_SIZE + 1,
                               offset_y + y*CELL_SIZE + 1,
                               CELL_SIZE-2, CELL_SIZE-2)
            if grid[y][x] == 1:
                pygame.draw.rect(screen, (0, 0, 255), rect)
            elif grid[y][x] == 2:
                pygame.draw.rect(screen, (255, 0, 0), rect)

def check_game_over(grid, human_turn):
    if human_turn:
        return not any(is_valid_human_move(grid, x, y)
                      for x in range(n) for y in range(GRID_ROWS-1))
    else:
        return not any(is_valid_computer_move(grid, x, y)
                      for y in range(GRID_ROWS) for x in range(n-1))

# Game loop
running = True
human_turn = True

while running:
    mouse_click = False
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.MOUSEBUTTONDOWN:
            mouse_click = True

    if not game_over:
        if human_turn:
            if mouse_click:
                mx, my = pygame.mouse.get_pos()
                gx = (mx - offset_x) // CELL_SIZE
                gy = (my - offset_y) // CELL_SIZE
                if 0 <= gx < n and 0 <= gy < GRID_ROWS:
                    if handle_human_move(grid, gx, gy):
                        human_turn = False
        else:
            if handle_computer_move(grid):
                human_turn = True
            
            if check_game_over(grid, human_turn):
                game_over = True
                winner_message = "You Win!" if human_turn else "You Lose!"
                end_time = pygame.time.get_ticks()

    if game_over and pygame.time.get_ticks() - end_time > 4000:
        running = False

    screen.fill((255, 255, 255))
    draw_grid(screen)
    
    if game_over:
        text = font.render(winner_message, True, (0, 128, 0))
        screen.blit(text, text.get_rect(center=(window_width//2, window_height//2)))
    
    pygame.display.flip()

pygame.quit()
